In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from configGAN import *
cfg = flying_objects_config()
import os

import tensorflow as tf
from tensorflow import keras
from utilsGAN import *
from sklearn.metrics import confusion_matrix
# import seaborn as sns
from datetime import datetime
import imageio
from skimage import img_as_ubyte

import pprint
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv3D, Conv2D, Conv1D, Convolution2D, Deconvolution2D, Cropping2D, UpSampling2D
from keras.layers import Input, Conv2DTranspose, ConvLSTM2D, TimeDistributed, GlobalMaxPooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers import Concatenate, concatenate, Reshape
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.layers import Input, merge
from keras.regularizers import l2
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout, LeakyReLU
import keras.backend as kb
from tensorflow.python.keras.engine import compile_utils
import io

In [3]:
def limit_gpu():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)
limit_gpu()

if cfg.GPU >=0:
    print("creating network model using gpu " + str(cfg.GPU))
    os.environ['CUDA_VISIBLE_DEVICES'] = str(cfg.GPU)
elif cfg.GPU >=-1:
    print("creating network model using cpu ")  
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
    os.environ["CUDA_VISIBLE_DEVICES"] = ""


creating network model using gpu 0


In [4]:
show_statistics(cfg.training_data_dir, fineGrained=False, title=" Training Data Statistics ")
show_statistics(cfg.validation_data_dir, fineGrained=False, title=" Validation Data Statistics ")
show_statistics(cfg.testing_data_dir, fineGrained=False, title=" Testing Data Statistics ")


######################################################################
##################### Training Data Statistics #####################
######################################################################
total image number 	 10817
total class number 	 3
class square 	 3488 images
class circular 	 3626 images
class triangle 	 3703 images
######################################################################

######################################################################
##################### Validation Data Statistics #####################
######################################################################
total image number 	 2241
total class number 	 3
class triangle 	 745 images
class square 	 783 images
class circular 	 713 images
######################################################################

######################################################################
##################### Testing Data Statistics #####################
##########################

In [5]:
batch_size = 64
image_shape = (cfg.IMAGE_HEIGHT, cfg.IMAGE_WIDTH, cfg.IMAGE_CHANNEL)

# Prepare dataset

# Prepare plots and logger

In [6]:
from improvedUtils import *

# Model Architecture

In [7]:
class architecture:
    __name__='Pix2Pix_model_v2'
    __changes__="Changed to a pix2pix model in order to test a greater network for generator and discriminator. Generator: U-Net, Discriminator: PatchGAN. Changed Discriminator optimizer to SGD"
    
    __normalization__='[-1,1]'
    
    generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    discriminator_optimizer = tf.keras.optimizers.SGD(learning_rate=2e-4)
    
    @staticmethod
    def discriminator():
        def downsample(filters, size, apply_batchnorm=True):
            initializer = tf.random_normal_initializer(0., 0.02)

            result = tf.keras.Sequential()
            result.add(tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                                     kernel_initializer=initializer, use_bias=False))

            if apply_batchnorm:
                result.add(tf.keras.layers.BatchNormalization())

            result.add(tf.keras.layers.LeakyReLU())

            return result
        
        def main():
            
            initializer = tf.random_normal_initializer(0., 0.02)

            inp = tf.keras.layers.Input(shape=image_shape, name='input_image')
            tar = tf.keras.layers.Input(shape=image_shape, name='target_image')

            x = tf.keras.layers.concatenate([inp, tar]) # (bs, 256, 256, channels*2)

            down1 = downsample(64, 4, False)(x) # (bs, 128, 128, 64)
            down2 = downsample(128, 4)(down1) # (bs, 64, 64, 128)
            down3 = downsample(256, 4)(down2) # (bs, 32, 32, 256)

            zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
            conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                        kernel_initializer=initializer,
                                        use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

            batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

            leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

            zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

            last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                        kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

            return tf.keras.Model(inputs=[inp, tar], outputs=last)
        
        
        return main()
    
    @staticmethod
    def generator():
        def downsample(filters, size, apply_batchnorm=True):
            initializer = tf.random_normal_initializer(0., 0.02)

            result = tf.keras.Sequential()
            result.add(tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                                     kernel_initializer=initializer, use_bias=False))

            if apply_batchnorm:
                result.add(tf.keras.layers.BatchNormalization())

            result.add(tf.keras.layers.LeakyReLU())

            return result
        
        def upsample(filters, size, apply_dropout=False):
            initializer = tf.random_normal_initializer(0., 0.02)

            result = tf.keras.Sequential()
            result.add(tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                            padding='same',
                                            kernel_initializer=initializer,
                                            use_bias=False))

            result.add(tf.keras.layers.BatchNormalization())

            if apply_dropout:
                result.add(tf.keras.layers.Dropout(0.5))

            result.add(tf.keras.layers.ReLU())

            return result

        def main():
            inputs = tf.keras.layers.Input(shape=image_shape)

            down_stack = [
                downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
                downsample(128, 4), # (bs, 64, 64, 128)
                downsample(256, 4), # (bs, 32, 32, 256)
                downsample(512, 4), # (bs, 16, 16, 512)
                downsample(512, 4), # (bs, 8, 8, 512)
                downsample(512, 4), # (bs, 4, 4, 512)
                downsample(512, 4), # (bs, 2, 2, 512)
                #downsample(512, 4), # (bs, 1, 1, 512)
            ]

            up_stack = [
                upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
                upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
                upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
                upsample(512, 4), # (bs, 16, 16, 1024)
                upsample(256, 4), # (bs, 32, 32, 512)
                upsample(128, 4), # (bs, 64, 64, 256)
                upsample(64, 4), # (bs, 128, 128, 128)
            ]

            initializer = tf.random_normal_initializer(0., 0.02)
            last = tf.keras.layers.Conv2DTranspose(3, 4,
                                                 strides=2,
                                                 padding='same',
                                                 kernel_initializer=initializer,
                                                 activation='tanh') # (bs, 256, 256, 3)

            x = inputs

            # Downsampling through the model
            skips = []
            for down in down_stack:
                x = down(x)
                skips.append(x)

            skips = reversed(skips[:-1])

            # Upsampling and establishing the skip connections
            for up, skip in zip(up_stack, skips):
                x = up(x)
                x = tf.keras.layers.Concatenate()([x, skip])

            x = last(x)

            return tf.keras.Model(inputs=inputs, outputs=x)
        return main()
    
    def loss():  # Decided by https://arxiv.org/abs/1611.07004
        loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
        LAMBDA = 100
        def generator_loss(disc_generated_output, gen_output, target): # https://arxiv.org/abs/1611.07004
            gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

            # mean absolute error
            l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

            total_gen_loss = gan_loss + (LAMBDA * l1_loss)

            return total_gen_loss, gan_loss, l1_loss

        def discriminator_loss(disc_real_output, disc_generated_output):
            real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

            generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

            total_disc_loss = real_loss + generated_loss

            return total_disc_loss

        return {'g_loss_fn':generator_loss, 'd_loss_fn':discriminator_loss}
    
    def __model__():
        return [architecture.generator, architecture.discriminator]

In [8]:
train_batch_generator, valid_batch_generator, test_batch_generator, nbr_train_data,nbr_valid_data, nbr_test_data = preprocess(image_shape, normalize_type=architecture.__normalization__)

train_x (30, 128, 128, 3) float32 -1.0 1.0
train_y (30, 128, 128, 3) float32 -1.0 1.0
{'BATCH_SIZE': 30,
 'DATA_AUGMENTATION': True,
 'DEBUG_MODE': True,
 'DROPOUT_PROB': 0.5,
 'GPU': 0,
 'IMAGE_CHANNEL': 3,
 'IMAGE_HEIGHT': 128,
 'IMAGE_WIDTH': 128,
 'LEARNING_RATE': 0.01,
 'LR_DECAY_FACTOR': 0.1,
 'NUM_EPOCHS': 200,
 'PRINT_EVERY': 50,
 'SAVE_EVERY': 1,
 'SEQUENCE_LENGTH': 10,
 'testing_data_dir': '../data/FlyingObjectDataset_10K/testing',
 'training_data_dir': '../data/FlyingObjectDataset_10K/training',
 'validation_data_dir': '../data/FlyingObjectDataset_10K/validation'}


# Model

In [9]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        #self.epoch = 0
    
    def special_compile(self, 
                d_optimizer=None, 
                g_optimizer=None,
                d_loss=None,
                g_loss=None,               
                loss_fn=None,
                metrics=None,
                loss_weights=None,
                weighted_metrics=None,
                run_eagerly=None,
                steps_per_execution=None,
              **kwargs):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss = d_loss
        self.g_loss = g_loss
        
        super().compile(metrics=metrics)
    
    def compile(self, **kwargs):
        raise NotImplementedError("Please use special_compile()")

    @tf.function
    def train_step(self, data): 
        #self.epoch += 1
        input_image, target = data # TODO: Must check if this iterates or take same image each run
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            
            # Generate images
            gen_output = self.generator(input_image, training=True)
            
            # Train discriminator
            disc_real_output = self.discriminator([input_image, target], training=True)
            disc_generated_output = self.discriminator([input_image, gen_output], training=True)
            
            # Training
            gen_total_loss, gen_gan_loss, gen_l1_loss = self.g_loss(disc_generated_output, gen_output, target)
            disc_loss = self.d_loss(disc_real_output, disc_generated_output)
            
            # Set weights
            generator_gradients = gen_tape.gradient(gen_total_loss,
                                              self.generator.trainable_variables)
            discriminator_gradients = disc_tape.gradient(disc_loss,
                                                   self.discriminator.trainable_variables)
            # Update weights
            self.g_optimizer.apply_gradients(zip(generator_gradients,
                                              self.generator.trainable_variables))
            self.d_optimizer.apply_gradients(zip(discriminator_gradients,
                                                  self.discriminator.trainable_variables))
            
        self.compiled_metrics.update_state(target, gen_output)
        
        met = {
                'gen_total_loss':gen_total_loss,
                'gen_gan_loss':gen_gan_loss,
                'gen_l1_loss':gen_l1_loss,
                'disc_loss':disc_loss, 
                
        }
        met.update({m.name: m.result() for m in self.metrics})
        return met
    

    def test_step(self, data):
        real_images, last_images = data
        valid, fake_last_frame = self(real_images, training=False)

        self.compiled_metrics.update_state(real_images, fake_last_frame)
            
        return {m.name: m.result() for m in self.metrics}
    
    def call(self, first_frame, training=False):
        fake_last_frame = self.generator(first_frame, training)
        validate_frame = self.discriminator([fake_last_frame, first_frame], training)
        
        return [validate_frame, fake_last_frame]

In [10]:
steps_per_epoch = (nbr_train_data // cfg.BATCH_SIZE) 
validation_steps=(nbr_valid_data//cfg.BATCH_SIZE)
log_dir = logger(architecture().__name__)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='epoch')
file_writer = tf.summary.create_file_writer(log_dir + "/plots/")

generator, discriminator =  architecture.__model__()
loss = architecture.loss()
model_to_json(discriminator(), log_dir + "/discriminator.json")
model_to_json(generator(), log_dir + "/generator.json")

gan = GAN(discriminator=discriminator(), generator=generator())

gan.special_compile(
    d_optimizer=architecture.discriminator_optimizer,
    g_optimizer=architecture.generator_optimizer,
    d_loss=loss['d_loss_fn'],
    g_loss=loss['g_loss_fn'],
    metrics=['accuracy', SSIM_loss]
)
gan.fit(
    x=train_batch_generator, 
    epochs=cfg.NUM_EPOCHS, 
    verbose=1, 
    batch_size=cfg.BATCH_SIZE,
    steps_per_epoch=steps_per_epoch, #
    validation_data=valid_batch_generator,
    validation_steps=validation_steps, 
    callbacks=[GANMonitor(num_img=3, validation_data=valid_batch_generator,log_dir=log_dir), tensorboard_callback],
    
) 


with open(log_dir+"/gan_finished", 'a') as f:
    f.write(architecture.__changes__)

Epoch 1/200
  1/360 [..............................] - ETA: 0s - gen_total_loss: 101.0597 - gen_gan_loss: 0.5846 - gen_l1_loss: 1.0048 - disc_loss: 1.8360 - accuracy: 0.1929 - SSIM_loss: 1.0017WARNING:tensorflow:From /home/stud/f/fremar16/miniconda3/envs/lab/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
360/360 [==============================] - 149s 415ms/step - gen_total_loss: 10.7761 - gen_gan_loss: 1.2574 - gen_l1_loss: 0.0952 - disc_loss: 1.3252 - accuracy: 0.2798 - SSIM_loss: 0.1359 - val_accuracy: 0.5330 - val_SSIM_loss: 0.0744
Epoch 2/200
360/360 [==============================] - 94s 261ms/step - gen_total_loss: 6.0705 - gen_gan_loss: 0.9959 - gen_l1_loss: 0.0507 - disc_loss: 1.3735 - accuracy: 0.3301 - SSIM_loss: 0.0777 - val_accuracy: 0.3134 - val_SSIM_loss: 0.0983
Epoch 3/200
360/